## Хакатон по распознаванию хот-догов. 
По мотивам крутейшего стартапа из сериала "кремниевая долина"!

https://www.youtube.com/watch?v=pqTntG1RXSY


Ваша задача: сделать революционный классификатор, который сразил большинство инвесторов и гиков из кремниевой долины не один раз!

Точнее, вам необходимо предсказать, есть на изображении хот-дог или нет. В файл с решением необходимо записать вероятность того, что на изображении **есть хот-дог**.

Качество модели будет измеряться с помощью метрики AUC-ROC, публичный лидерборд (рейтинговая таблица соревнования на платформе kaggle) будет строиться по 50% наблюдений. 

Чуть подробнее про публичный и приватный лидерборд: вам дана тестовая выборка, для которой неизвестна целевая переменная. Вы обучаете модель, предсказываете для тестовой выборки, формируете из предсказаний csv-файл, и загружаете его на платформу kaggle. Видите значение метрики AUC-ROC. Это значение называется метрикой на публичном лидерборде, и считается оно не по всем данным, а лишь по 50% от тестовой выборки. Когда соревнование заканчивается, AUC-ROC пересчитывается уже для 100% тестовой выборки, и это уже называется приватный лидерборд, он же - финальный.

Такой подход является классическим для большей части соревнований по анализу данных, и основан он на том, что если не делать такого разделения, то будет происходить неявное переобучение модели под тестовую выборку. 

На этом вводная часть заканчивается, и мы искренне желаем вам удачи :) <br>
Сможете ли вы повторить успех Jian Yuang'a? Мы в вас верим!

<hr>

In [ ]:
import os

import numpy as np
import pandas as pd

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

from matplotlib.image import imread
import matplotlib.pyplot as plt
%matplotlib inline

import PIL

In [ ]:
import os
#print(os.listdir("../input/train/train/"))
print(os.listdir("../input/train2/train2/train2/"))

Разархивируем файлы с данными и импортируем необходимые библиотеки

In [ ]:
#!unzip -q -o test.zip
#!unzip -q -o train.zip

In [ ]:
font = {
    'family': 'serif',
    'color':  'darkred',
    'weight': 'bold',
    'size': 22,
}
SEED = 257
#TRAIN_DIR = '../input/train/train/'
TRAIN_DIR = '../input/train2/train2/train2/'
TEST_DIR = '../input/test/test/'
categories = ['hot dog', 'not hot dog']

image_width = 100
image_height = 100
X, y = [], []
for category in categories:
    category_dir = os.path.join(TRAIN_DIR, category)
    for image_path in os.listdir(category_dir):
        if not (image_path.endswith('.png')):
            continue 
        image = PIL.Image.open(os.path.join(category_dir, image_path))#.convert("L")
        if (image.width == image_width) and (image.height == image_height):
            X.append(np.array(image))
        if category=='hot dog': 
            y.append(1) 
        else: 
            y.append(0)
        #y.append(category)
        
X_array = np.array(X)#.reshape(len(X), image_width, image_height, 1)
Y_array = np.array(y) 
#Y_array_encoded = [1 if x == 'hot dog' else 0 for x in Y_array]
from tensorflow.keras.utils import to_categorical
Y_array_encoded = to_categorical(Y_array)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(np.array(X_array), np.array(Y_array_encoded), test_size=0.25, random_state=SEED)
#X_train = np.array(X_array)
#y_train = np.array(Y_array_encoded)

from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    #featurewise_center=True,
    #featurewise_std_normalization=True,
    #rescale=1./255,
    rotation_range=180,
    #width_shift_range=.05,
    #height_shift_range=.05,
    horizontal_flip=True,
    vertical_flip=True)

# training the image preprocessing
# compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied)
datagen.fit(X_train, augment=True)

In [ ]:
#X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras import optimizers

model = Sequential()
model.add(Convolution2D(32, kernel_size = (3, 3), activation='relu', input_shape=(image_width, image_height, 3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Convolution2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Convolution2D(96, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Convolution2D(96, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Convolution2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
#model.add(Dropout(0.3))
model.add(Dense(2, activation = 'softmax'))

#model = Sequential()
#model.add(Convolution2D(64, kernel_size=5, activation='sigmoid'))
#model.add(MaxPooling2D(pool_size=2))
#model.add(Convolution2D(32, kernel_size=3, activation='sigmoid'))
#model.add(MaxPooling2D(pool_size=2))
#model.add(Flatten())
##model.add(Dropout(0.5))
#model.add(Dense(2,activation = 'softmax'))
#model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

#adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) #точность составила 0.92127
#model.compile(optimizer=keras.optimizers.Adadelta(), loss='binary_crossentropy', metrics=['accuracy']) #точность составила 0.5

model.summary()

batch_size = 64
epochs = 50
# fits the model on batches with real-time data augmentation:
model.fit_generator(datagen.flow(X_train, y_train, batch_size=batch_size),steps_per_epoch = X_train.shape[0]//batch_size, epochs=epochs)

In [ ]:
model.fit(X_train, y_train, 
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(X_test, y_test))

In [ ]:
#prediction = model.predict_classes(X_test)
#roc_auc_score(y_test[:,1], prediction)

In [ ]:
X_control, y_control = [], []
leaderboard_filenames=[]
category_dir = os.path.join(TEST_DIR)
for image_path in os.listdir(category_dir):
    if not (image_path.endswith('.png')):
        continue 
    image = PIL.Image.open(os.path.join(category_dir, image_path))#.convert("L")
    if (image.width == image_width) and (image.height == image_height):
        X_control.append(np.array(image))    
        leaderboard_filenames.append(image_path)
X_control_array = np.array(X_control)#.reshape(len(X_control), image_width, image_height, 1)

In [ ]:
leadeboard_predictions = model.predict_proba(X_control_array)[:,1]

In [ ]:
#leadeboard_predictions[leadeboard_predictions>0]

In [ ]:
"""
idx = 467

plt.axis("off");
if leadeboard_predictions[idx] > 0.5:
    plt.text(20, -5, 'HOT DOG!!!', fontdict=font)
else:
    plt.text(15, -5,'not hot dog...', fontdict=font)
plt.imshow(leaderboard_X[idx]);
"""

In [ ]:
submission = pd.DataFrame(
    {
        'image_id': leaderboard_filenames, 
        'image_hot_dog_probability': leadeboard_predictions
    }
)

In [ ]:
submission.to_csv('submit_net_daug4.csv', index=False)